In [394]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

### Read in Data

In [395]:
df=pd.read_excel("CoffeeData.xlsx")
df.head()

,Week,Share,Outside,Brand 1,Brand 2,Brand 3,Brand 4,Price,Feature,Display,F&D,Spot 1,Spot 2,Spot 3,Spot 4,Spot 5,Spot 6
0,1,0.005929,0.969501,1,0,0,0,4.29168,0.000000,0.000000,0.0,134,146,169,170,164,168
1,1,0.000732,0.969501,0,1,0,0,3.39696,0.000000,0.000000,0.0,134,146,169,170,164,168
2,1,0.016865,0.969501,0,0,1,0,2.93088,42.261265,0.000000,0.0,134,146,169,170,164,168
3,1,0.006973,0.969501,0,0,0,1,3.40096,0.000000,1.492933,0.0,134,146,169,170,164,168
4,2,0.006089,0.967560,1,0,0,0,4.29168,0.000000,0.000000,0.0,138,141,175,164,157,176


#### 1. Simple OLS

In [396]:
Y=df["Share"]/df["Outside"]
X=df[["Brand 1", "Brand 2", "Brand 3", "Brand 4",
     "Price", "Feature", "Display", "F&D"]]
res1=sm.OLS(Y,X).fit()
print(res1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.575
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     86.48
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           3.92e-79
Time:                        08:20:26   Log-Likelihood:                 1756.4
No. Observations:                 456   AIC:                            -3497.
Df Residuals:                     448   BIC:                            -3464.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Brand 1        0.0117      0.003      3.810      0.0

#### 2. 2SLS by Statsmodel

In [397]:
# First Stage: price on X_jt, intercepts, and instruments
X1=df[["Brand 1", "Brand 2", "Brand 3", "Brand 4",
       "Feature", "Display", "F&D",
       "Spot 1", "Spot 2", "Spot 3",
       "Spot 4", "Spot 5", "Spot 6"]]
resf=sm.OLS(df["Price"],X1).fit()
print(resf.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     298.4
Date:                Wed, 20 Feb 2019   Prob (F-statistic):          1.47e-203
Time:                        08:20:26   Log-Likelihood:                -90.276
No. Observations:                 456   AIC:                             206.6
Df Residuals:                     443   BIC:                             260.1
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Brand 1        3.7742      0.073     51.935      0.0

In [398]:
# Second stage: replace p_jt in X_jt with predicted p_jt values
df["Price_pred"]=resf.predict()
X2=df[["Brand 1", "Brand 2", "Brand 3", "Brand 4", 
       "Price_pred", "Feature", "Display", "F&D"]]
ress=sm.OLS(Y,X2).fit()
print(ress.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     86.94
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           1.99e-79
Time:                        08:20:26   Log-Likelihood:                 1757.1
No. Observations:                 456   AIC:                            -3498.
Df Residuals:                     448   BIC:                            -3465.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Brand 1        0.0159      0.005      3.430      0.0

#### 3. 2SLS by matrix computation

In [399]:
nudta=np.matrix(df[["Brand 1", "Brand 2", "Brand 3", "Brand 4", 
                    "Price", "Feature", "Display", "F&D"]])
oins=np.matrix(df[["Brand 1", "Brand 2", "Brand 3", "Brand 4", 
                    "Spot 1", "Spot 2", "Spot 3",
                    "Spot 4", "Spot 5", "Spot 6",
                    "Feature", "Display", "F&D"]])
#Coefficients
mmm=np.linalg.inv((oins.T)*oins)
sha=np.matrix(Y)
bb=(np.linalg.inv(nudta.T*oins*mmm*oins.T*nudta))*(nudta.T*oins*mmm*oins.T*sha.T)

#Std err
erbb=sha.T-nudta*bb
vcvbb=np.array(erbb.T*erbb)[0]*np.array(np.linalg.inv(nudta.T*oins*mmm*oins.T*nudta))/df.shape[0]
se=np.sqrt(vcvbb.diagonal())

pd.DataFrame({"Coef":bb.A1,"S.E":se},index=["Brand 1", "Brand 2", "Brand 3", "Brand 4", 
                    "Price_pred", "Feature", "Display", "F&D"])

,Coef,S.E
Brand 1,0.015931,0.004621
Brand 2,0.006305,0.002754
Brand 3,0.020768,0.002930
Brand 4,0.009177,0.003098
Price_pred,-0.001758,0.000942
Feature,0.000038,0.000011
Display,0.000046,0.000033
F&D,0.000037,0.000042


### BLP Model 

In [428]:
def contraction(table):
    '''
    Function to compute δ to match s_jt and S_jt using contraction mapping
    
    Input: ∆µ_djt 
    Output: δ_jt
    
    This method is similar to policy function iteration in dynamic programming.
    '''
    delta=Y # Initialize δ_jt
    C=100
    tol=1e-15
    newtab=table
    iteration=1
    maxiteration=100
    
    while C>tol and iteration<maxiteration:

        for i in range(D):
            exp1=np.exp(delta*df['Brand 1']+table[i]*df['Brand 1'])
            exp2=np.exp(delta*df['Brand 2']+table[i]*df['Brand 2'])
            exp3=np.exp(delta*df['Brand 3']+table[i]*df['Brand 3'])
            exp4=np.exp(delta*df['Brand 4']+table[i]*df['Brand 4'])
            denom=1+exp1+exp2+exp3+exp4
            newtab[i]=(exp1*df['Brand 1']+exp2*df['Brand 2']+exp3*df['Brand 3']+exp4*df['Brand 4'])/denom
        Sjt=np.mean(newtab,axis=1)
        deltanew=delta+np.log(Y)-np.log(Sjt)
        C=max(abs(deltanew-delta))
        #print('distance= ', C) #print C to see C onverges
        delta=deltanew
        iteration+=1
    return delta

In [425]:
def residuals(delta):
    '''
    Function to get criterion value of ξjt by running 2SLS
    
    Input: δ_jt from contraction function
    Output: ξ'*Z*inv(Z'Z)*Z'*ξ
    '''
    # First Stage: price on X_jt, intercepts, and instruments
    X1=df[["Brand 1", "Brand 2", "Brand 3", "Brand 4",
           "Feature", "Display", "F&D",
           "Spot 1", "Spot 2", "Spot 3",
           "Spot 4", "Spot 5", "Spot 6"]]
    resf=sm.OLS(df["Price"],X1).fit()
    
    # Second stage: replace p_jt in X_jt with predicted p_jt values
    df["Price_pred"]=resf.predict()
    X2=df[["Brand 1", "Brand 2", "Brand 3", "Brand 4", 
           "Price_pred", "Feature", "Display", "F&D"]]
    global ress
    ress=sm.OLS(delta,X2).fit()
    #print(ress.summary())
    residual=np.matrix(delta-ress.predict())
    return residual*oins*np.linalg.inv(oins.T*oins)*oins.T*residual.T

In [501]:
D=50 #Number of random draws
np.random.seed(100)
draw=np.random.uniform(0,5,36*D).reshape(D,36)
len(draw[0])

36

In [503]:
cri=10 #Initial criterion value 
diff=10
tol=1e-20 #Tolerance amount
itera=1
maxitera=50
while diff>tol and itera<maxitera:
    #Initial values of elements in Γ (iterate through each row in the generated draw matrix)
    a11,a12,a13,a14,a15,a16,a17,a18,b22,b23,b24,b25,b26,b27,b28,c33,c34,c35,c36,c37,c38,d44,d45,d46,d47,d48,e55,e56,e57,e58,f66,f67,f68,g77,f78,h88=draw[itera]
    r=np.matrix([[a11,a12,a13,a14,a15,a16,a17,a18], 
                [0,b22,b23,b24,b25,b26,b27,b28],
                [0,0,c33,c34,c35,c36,c37,c38],
                [0,0,0,d44,d45,d46,d47,d48],
                [0,0,0,0,e55,e56,e57,e58],
                [0,0,0,0,0,f66,f67,f68],
                [0,0,0,0,0,0,g77,f78],
                [0,0,0,0,0,0,0,h88], ])
    # Make draws from the distribution to get ∆Θd
   
    Q=np.random.normal(0,1,D*8).reshape(D,8)
    QGT=Q@r
    
    # Compute ∆µ_djt = ∆α_dj + X_jt*∆β_d (Part that depend on i)
    table=pd.DataFrame({"Week":df["Week"]})
    
    for i in range(D):
        #coef elements refer to a1,a2,a3,a4,bp,bf,bd,bfd
        coef=QGT[i].tolist()[0] #Get coefficients in the ith simulation
        ev1=coef[0]+coef[4]*df['Price']+coef[5]*df['Feature']+coef[6]*df['Display']+coef[7]*df['F&D']
        ev2=coef[1]+coef[4]*df['Price']+coef[5]*df['Feature']+coef[6]*df['Display']+coef[7]*df['F&D']
        ev3=coef[2]+coef[4]*df['Price']+coef[5]*df['Feature']+coef[6]*df['Display']+coef[7]*df['F&D']
        ev4=coef[3]+coef[4]*df['Price']+coef[5]*df['Feature']+coef[6]*df['Display']+coef[7]*df['F&D']

        table[i]=ev1*df['Brand 1']+ev2*df['Brand 2']+ev3*df['Brand 3']+ev4*df['Brand 4']
    table=table.drop(columns=['Week'])
    
    # Compute δ_jt (Part that doesn't depend on i)
    delta_y=contraction(table)
 
    # Compute ξjt using criterion ξ'*Z*inv(Z'Z)*Z'*ξ
    cri_new=residuals(delta_y)
    diff=abs(cri-cri_new)
    cri=cri_new
    
    print("Iteration",itera,diff)
    itera+=1


C:\Users\lliu9\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp
C:\Users\lliu9\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp
C:\Users\lliu9\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp
C:\Users\lliu9\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in exp


Iteration 1 [[8.88797034]]
Iteration 2 [[2.22044605e-16]]
Iteration 3 [[4.4408921e-16]]
Iteration 4 [[2.66453526e-15]]
Iteration 5 [[3.77475828e-15]]
Iteration 6 [[3.77475828e-15]]
Iteration 7 [[1.33226763e-15]]
Iteration 8 [[1.33226763e-15]]
Iteration 9 [[8.8817842e-16]]
Iteration 10 [[1.33226763e-15]]
Iteration 11 [[1.77635684e-15]]
Iteration 12 [[4.4408921e-16]]
Iteration 13 [[1.33226763e-15]]
Iteration 14 [[8.8817842e-16]]
Iteration 15 [[1.33226763e-15]]
Iteration 16 [[6.66133815e-16]]
Iteration 17 [[5.32907052e-15]]
Iteration 18 [[2.22044605e-15]]
Iteration 19 [[4.88498131e-15]]
Iteration 20 [[7.77156117e-15]]
Iteration 21 [[5.55111512e-15]]
Iteration 22 [[0.]]


### Coefficients and S.E. corresponding to the Γ with Statsmodel


In [504]:
#Coefficients and S.E. corresponding to the Γ.
print(ress.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                     447.2
Date:                Wed, 20 Feb 2019   Prob (F-statistic):          1.17e-197
Time:                        19:22:53   Log-Likelihood:                -117.29
No. Observations:                 456   AIC:                             250.6
Df Residuals:                     448   BIC:                             283.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Brand 1       -2.5325      0.283     -8.941      0.0

### Coefficients and S.E. corresponding to the Γ with Matrix Multiplication


In [505]:
#Coefficients
mmm=np.linalg.inv((oins.T)*oins)
sha=np.matrix(delta.fillna(np.mean(delta)))
bb=(np.linalg.inv(nudta.T*oins*mmm*oins.T*nudta))*(nudta.T*oins*mmm*oins.T*sha.T)

#Std err
erbb=sha.T-nudta*bb
vcvbb=np.array(erbb.T*erbb)[0]*np.array(np.linalg.inv(nudta.T*oins*mmm*oins.T*nudta))/(df.shape[0]-8)
se=np.sqrt(vcvbb.diagonal())

pd.DataFrame({"Coef":bb.A1,"S.E":se},index=["Brand 1", "Brand 2", "Brand 3", "Brand 4", 
                    "Price_pred", "Feature", "Display", "F&D"])

,Coef,S.E
Brand 1,-2.555663,0.375150
Brand 2,-4.225686,0.223569
Brand 3,-2.599178,0.237861
Brand 4,-3.485292,0.251512
Price_pred,-0.222330,0.076453
Feature,0.003229,0.000925
Display,0.020049,0.002640
F&D,-0.017658,0.003398
